## Machine Learning Pipeline - Sarcasm Detection
- Author: Naufal Rahfi Anugerah

## Import Required Library

In [27]:
import tensorflow as tf
import tensorflow_transform as tft

import json

import pandas as pd
import zipfile as zf
import os

from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.dsl.components.common.resolver import Resolver 

from tfx.types.standard_artifacts import Model, ModelBlessing
from tfx.types import Channel 

from tfx.components import Tuner
from tfx.components import Evaluator

from tfx.proto import trainer_pb2
from tfx.proto import example_gen_pb2

## Data Configuration

### Download Dataset

In [28]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

!mkdir raw

Dataset URL: https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection
License(s): Attribution 4.0 International (CC BY 4.0)




  0%|          | 0.00/3.30M [00:00<?, ?B/s]
 30%|███       | 1.00M/3.30M [00:01<00:02, 913kB/s]
 61%|██████    | 2.00M/3.30M [00:01<00:00, 1.60MB/s]
 91%|█████████ | 3.00M/3.30M [00:01<00:00, 2.17MB/s]
100%|██████████| 3.30M/3.30M [00:01<00:00, 1.91MB/s]
A subdirectory or file raw already exists.


In [29]:
!move news-headlines-dataset-for-sarcasm-detection.zip raw/

        1 file(s) moved.


### Extract File

In [30]:
files = "raw/news-headlines-dataset-for-sarcasm-detection.zip"
zip = zf.ZipFile(files, 'r')
zip.extractall('raw/')
zip.close()

### Data Converting

In [31]:
import json
import pandas as pd

!mkdir data
# Path to the JSON files
file_paths = [r"raw\Sarcasm_Headlines_Dataset_v2.json", r"raw\Sarcasm_Headlines_Dataset_v2.json"]
# Initialize a list to store the data
data_list = []
# Read and process each line as a separate JSON object
for file_path in file_paths:
    with open(file_path, 'r') as file:
        for line in file:
            data_list.append(json.loads(line))

# Normalize JSON data
df = pd.json_normalize(data_list)

A subdirectory or file data already exists.


### Export Data

In [32]:
df.to_csv("raw/data.csv", index=False)

### Data Loading

In [33]:
dataset = pd.read_csv("raw/data.csv")

In [34]:
dataset.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clock of hair loss,https://www.theonion.com/thirtysomething-scientists-unveil-doomsday-clock-of-hai-1819586205
1,0,"dem rep. totally nails why congress is falling short on gender, racial equality",https://www.huffingtonpost.com/entry/donna-edwards-inequality_us_57455f7fe4b055bb1170b207
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-veggies-9-delici_b_8899742.html
3,1,inclement weather prevents liar from getting to work,https://local.theonion.com/inclement-weather-prevents-liar-from-getting-to-work-1819576031
4,1,mother comes pretty close to using word 'streaming' correctly,https://www.theonion.com/mother-comes-pretty-close-to-using-word-streaming-cor-1819575546


### Data Assesing

In [35]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57238 entries, 0 to 57237
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  57238 non-null  int64 
 1   headline      57238 non-null  object
 2   article_link  57238 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [36]:
def data_assesing(data):
    print(f"Total NaN/Null Data per Column:\n{data.isna().sum().sort_values(ascending=False)}\n")
    print(f"Data Shape:\n{data.shape}")
    print(f"\nTotal Duplicated Data: {data.duplicated().sum()}")

data_assesing(dataset)

Total NaN/Null Data per Column:
is_sarcastic    0
headline        0
article_link    0
dtype: int64

Data Shape:
(57238, 3)

Total Duplicated Data: 28621


### Data Cleaning

In [37]:
dataset = pd.DataFrame(dataset.drop(columns=["article_link"]))
dataset = pd.DataFrame(dataset.drop_duplicates())

In [38]:
data_assesing(dataset)

Total NaN/Null Data per Column:
is_sarcastic    0
headline        0
dtype: int64

Data Shape:
(28503, 2)

Total Duplicated Data: 0


## Export Data

In [39]:
# Write to CSV
dataset.to_csv('data/data.csv', index=False)

## Set Variables

In [40]:
PIPELINE_NAME = "sarcasm-pipeline"
SCHEMA_PIPELINE_NAME = "sarcasm-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

In [41]:
DATA_ROOT = "data"

In [42]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Data Ingestion

In [43]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [44]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 5
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## Data Validation

### Create Statistics Summary

In [45]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 6
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [46]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Create Data Schema

In [47]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 7
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [48]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'headline',BYTES,required,,-
'is_sarcastic',INT,required,,-


### Create Example Validator

In [49]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 8
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

### Checking Anomalies in Data

In [50]:
interactive_context.show(example_validator.outputs['anomalies'])

## Data Preprocessing

In [51]:
TRANSFORM_MODULE_FILE = "transform.py"

In [52]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf

LABEL_KEY = "is_sarcastic"
FEATURE_KEY = "headline"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}
    
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

Writing transform.py


In [53]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: pipelines\sarcasm-pipeline\Transform\transform_graph\9\.temp_path\tftransform_tmp\19b30806d57140f9be8c730e06d5d106\assets


INFO:tensorflow:Assets written to: pipelines\sarcasm-pipeline\Transform\transform_graph\9\.temp_path\tftransform_tmp\19b30806d57140f9be8c730e06d5d106\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 9
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Model Development

### Model Tuner

In [59]:
TUNER_MODULE_FILE = "tuner.py"

In [65]:
%%writefile {TUNER_MODULE_FILE}
import keras_tuner as kt
import tensorflow as tf
import tensorflow_transform as tft
from typing import NamedTuple, Dict, Text, Any
from keras_tuner.engine import base_tuner
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "is_sarcastic"
FEATURE_KEY = "headline"
NUM_EPOCHS = 2

TunerFnResult = NamedTuple("TunerFnResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any]),
])

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_binary_accuracy",
    mode="max",
    verbose=1,
    patience=10,
)


def transformed_name(key):
    return f"{key}_xf"


def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64):
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )
    
    return dataset

def model_builder(hypa, vectorizer_layer):

  # Define hyperparameters (consider reducing the range if needed)
  hidden_layers = hypa.Choice("hidden_layers", values=[1, 2])
  embedding_size = hypa.Int("embedding_size", min_value=32, max_value=128, step=32)
  lstm_units = hypa.Int("lstm_units", min_value=32, max_value=128, step=32)
  dropout_rate = hypa.Float("dropout_rate", min_value=0.2, max_value=0.5, step=0.1)
  dense_units = hypa.Int("dense_units", min_value=32, max_value=128, step=32)  # New hyperparameter

  # Model construction
  inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)

  x = vectorizer_layer(inputs)
  x = layers.Embedding(5000, embedding_size)(x)
  x = layers.Bidirectional(layers.LSTM(lstm_units))(x)

  # Add dense layers based on hyperparameter
  for _ in range(hidden_layers):
    x = layers.Dense(dense_units, activation="relu")(x)  # Use dense_units hyperparameter
    x = layers.Dropout(dropout_rate)(x)

  outputs = layers.Dense(1, activation="sigmoid")(x)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  # Compile for binary classification (consider learning rate tuning)
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Consider hyperparameter tuning
      loss="binary_crossentropy",
      metrics=["accuracy"],
  )

  return model


def tuner_fn(fn_args):
  # Load the transform graph
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

  # Create training and evaluation datasets with transformed features
  train_set = input_fn(fn_args.train_files[0], tf_transform_output, NUM_EPOCHS)
  eval_set = input_fn(fn_args.eval_files[0], tf_transform_output, NUM_EPOCHS)

  # Extract text features for vectorization
  vectorizer_dataset = train_set.map(lambda f, l: f[transformed_name(FEATURE_KEY)])

  # Define and adapt the text vectorization layer
  vectorizer_layer = layers.TextVectorization(
      max_tokens=1000, output_mode="int", output_sequence_length=100
  )
  vectorizer_layer.adapt(vectorizer_dataset)

  # Create a Hyperband tuner with the vectorization layer
  tuner = kt.Hyperband(
      hypermodel=lambda hypa: model_builder(hypa, vectorizer_layer),
      objective=kt.Objective("accuracy", direction="max"),
      max_epochs=NUM_EPOCHS,
      factor=3,
      directory=fn_args.working_dir,
      project_name="kt_hyperband",
  )

  # Define fit arguments with early stopping callback
  fit_kwargs = {
      "callbacks": [early_stopping_callback],
      "x": train_set,
      "validation_data": eval_set,
      "steps_per_epoch": fn_args.train_steps,
      "validation_steps": fn_args.eval_steps,
  }

  return TunerFnResult(tuner=tuner, fit_kwargs=fit_kwargs)

Overwriting tuner.py


In [66]:
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=200),
)
interactive_context.run(tuner)

Trial 2 Complete [00h 05m 08s]
accuracy: 0.8028839230537415

Best accuracy So Far: 0.8145952224731445
Total elapsed time: 00h 05m 37s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in pipelines\sarcasm-pipeline\.temp\11\kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
hidden_layers: 2
embedding_size: 32
lstm_units: 32
dropout_rate: 0.30000000000000004
dense_units: 128
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8145952224731445
Trial summary
Hyperparameters:
hidden_layers: 2
embedding_size: 128
lstm_units: 128
dropout_rate: 0.2
dense_units: 32
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8028839230537415


ExecutionResult(
    component_id: Tuner
    execution_id: 11
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

### Training Model

In [67]:
TRAINER_MODULE_FILE = "trainer.py"

In [68]:
%%writefile {TRAINER_MODULE_FILE}
import os
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_hub as hub
from tensorflow.keras import layers
from tfx.components.trainer.fn_args_utils import FnArgs


LABEL_KEY = "is_sarcastic"
FEATURE_KEY = "headline"


def transformed_name(key): 
    return f"{key}_xf"


def gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=64):
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy()
    )
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY),
    )
    
    return dataset

def model_builder(vectorizer_layer, hp):
    inputs = tf.keras.Input(
        shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string
    )

    x = vectorizer_layer(inputs)
    x = layers.Embedding(input_dim=5000, output_dim=hp["embedding_size"])(x)
    x = layers.Bidirectional(layers.LSTM(hp["lstm_units"]))(x)

    for _ in range(hp["hidden_layers"]):
        x = layers.Dense(hp["dense_units"], activation=tf.nn.relu)(x)
        x = layers.Dropout(hp["dropout_rate"])(x)

    outputs = layers.Dense(1, activation=tf.nn.sigmoid)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()],
    )
    
    model.summary()
    
    return model

def _get_serve_tf_example_fn(model, tf_transform_output):
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(
            serialized_tf_examples, feature_spec
        )
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using transformed features
        return model(transformed_features)
    
    return serve_tf_examples_fn


def run_fn(fn_args: FnArgs):
  # Extract hyperparameters
  hyperparameters = fn_args.hyperparameters["values"]

  # Define log directory and callbacks
  log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), "logs")
  callbacks = [
      tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq="batch"),
      tf.keras.callbacks.EarlyStopping(
          monitor="val_binary_accuracy", mode="max", verbose=1, patience=10
      ),
      tf.keras.callbacks.ModelCheckpoint(
          fn_args.serving_model_dir,
          monitor="val_binary_accuracy",
          mode="max",
          verbose=1,
          save_best_only=True,
      ),
  ]

  # Load transform graph and create datasets
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
  train_set = input_fn(fn_args.train_files, tf_transform_output, hyperparameters["tuner/epochs"])
  eval_set = input_fn(fn_args.eval_files, tf_transform_output, hyperparameters["tuner/epochs"])
  vectorizer_dataset = train_set.map(lambda f, l: f[transformed_name(FEATURE_KEY)])

  # Define and adapt vectorization layer
  vectorizer_layer = layers.TextVectorization(
      max_tokens=5000, output_mode="int", output_sequence_length=500
  )
  vectorizer_layer.adapt(vectorizer_dataset)

  # Build model using hyperparameters
  model = model_builder(vectorizer_layer, hyperparameters)

  # Train the model with callbacks
  model.fit(
      x=train_set,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_set,
      validation_steps=fn_args.eval_steps,
      callbacks=callbacks,
      epochs=hyperparameters["tuner/epochs"],
      verbose=1,
  )

  # Define serving signature and save the model
  signatures = {
      "serving_default": _get_serve_tf_example_fn(model, tf_transform_output).get_concrete_function(
          tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
      )
  }
  model.save(fn_args.serving_model_dir, save_format="tf", signatures=signatures)

Overwriting trainer.py


In [69]:
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=["train"], num_steps=500),
    eval_args=trainer_pb2.EvalArgs(splits=["eval"], num_steps=200),
)
interactive_context.run(trainer)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 headline_xf (InputLayer)    [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 500)              0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 500, 32)           160000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0   


Epoch 1: val_binary_accuracy improved from -inf to 0.81030, saving model to pipelines\sarcasm-pipeline\Trainer\model\12\Format-Serving


INFO:tensorflow:Assets written to: pipelines\sarcasm-pipeline\Trainer\model\12\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\sarcasm-pipeline\Trainer\model\12\Format-Serving\assets


500/500 [==============================] - 97s 187ms/step - loss: 0.5807 - binary_accuracy: 0.6810 - val_loss: 0.4152 - val_binary_accuracy: 0.8103
Epoch 2/2
214/500 [===========>..................] - ETA: 2:17 - loss: 0.3580 - binary_accuracy: 0.8431WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 1000 batches). You may need to use the repeat() function when building your dataset.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\sarcasm-pipeline\Trainer\model\12\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\sarcasm-pipeline\Trainer\model\12\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 12
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

## Analysis and Validation Model

In [71]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing),
).with_id("Latest_blessed_model_resolver")

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 13
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [74]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='is_sarcastic')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

## Evaluation Model

In [75]:
evaluator = Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_config
)
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 15
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [76]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

In [77]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/sarcasm-detection-model'))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 16
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))